<img src="img/mmselfsup_logo.png">

# 自监督预训练模型的评估：“检测” 下游任务

<a href="https://colab.research.google.com/github/wangruohui/OpenMMLabCourse/blob/main/codes/MMSelfSup_tutorials/【5】自监督预训练模型的评估：“检测”下游任务.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**MMSelfSup Repo**：[https://github.com/open-mmlab/mmselfsup](https://github.com/open-mmlab/mmselfsup)

**MMSelfSup 官方文档链接**：[https://mmselfsup.readthedocs.io/en/latest](https://mmselfsup.readthedocs.io/en/latest)

**作者**：OpenMMLab

## 0. 任务介绍

“检测” 下游任务评估的 **`基本流程`** 如下：
1. 将自监督预训练好的 backbone 权值迁移到下游 “检测” 任务的 backbone 上
2. 使用下游任务（检测）的数据集对下游任务的模型进行微调，以此验证自监督预训练模型提取图片特征的效果。

在使用 “检测” 下游任务对自监督预训练模型进行评估中，MMSelfSup 目前支持了以下几种评估配置：（详细请参考官方[文档](https://mmselfsup.readthedocs.io/zh_CN/latest/tutorials/6_benchmarks.html#id5)）
+ 将自监督预训练的权值迁移到 `Faster R-CNN` 模型上，并在 `Pascal VOC 2007 + 2012` 数据集上进行评估
+ 将自监督预训练的权值迁移到 `Mask R-CNN` 模型上，并在 `COCO2017` 数据集上进行评估

本教程将演示第一种评估配置。

**注意：下游任务的 backbone 必须和自监督预训练模型的 backbone 保持一致，才能进行自监督预训练权值的迁移！**

## 1. 环境配置

### 1.1 查看 Python、PyTorch 和 Torchvision 的版本

In [ ]:
# Check nvcc version
!nvcc -V

In [ ]:
# Check GCC version
!gcc --version

In [ ]:
# Check PyTorch installation
import torch, torchvision
print(torch.__version__)
print(torch.cuda.is_available())

### 1.2 安装 MMSelfSup 的依赖库：MMCV

In [ ]:
!pip install openmim

In [ ]:
!mim install mmcv

### 1.3  安装 MMSelfSup

In [ ]:
%cd /content

In [ ]:
!git clone https://github.com/open-mmlab/mmselfsup.git
%cd /content/mmselfsup

In [ ]:
# Install MMSelfSup from source
!pip install -e . 

### 1.4 安装 MMDetection 
在这里，mmdet 是我们这里所需要用到的依赖库，所以直接用 `pip install mmdet` 命令安装即可。

其他方式的安装详情请参考： [MMDetection 文档](https://github.com/open-mmlab/mmdetection/blob/master/docs/en/get_started.md)。

In [ ]:
pip install mmdet

### 1.5 检查安装是否正确

In [ ]:
import mmselfsup
print(mmselfsup.__version__)

import mmdet
print(mmdet.__version__)

## 2. 准备数据集

### 2.0 数据集介绍

本教程将在 `Pascal VOC 2012` 和 `Pascal VOC 2007` 数据集上对自监督预训练好的模型进行“检测”下游任务的评估。

Pascal VOC 2007 数据集包含 20 个类别，Pascal VOC 2012 在此基础上进行无交集扩展（即类别相同，图片不同）。

在本教程中，我们根据 MOCO 论文里的数据集配置进行训练和测试：使用 VOC 07 和 VOC 12 的训练集和验证集进行微调，在 VOC 07 数据集上进行测试。

数据集官方网址：http://host.robots.ox.ac.uk/pascal/VOC/

### 2.1 下载数据集

In [ ]:
%cd /content/mmselfsup

In [ ]:
!mkdir data
%cd data

!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar

!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar

### 2.2 解压数据集

In [ ]:
!tar -xf VOCtrainval_11-May-2012.tar
!tar -xf VOCtrainval_06-Nov-2007.tar
!tar -xf VOCtest_06-Nov-2007.tar

## 3. 准备自监督预训练模型的 backbone 权值文件

### 3.1 针对自监督预训练过程中保存的 ckeckpoint 文件

我们使用第一个教程 `模型自监督预训练 之 SimCLR` 中训练保存下来的 `epoch_1.pth` 文件进行演示，该文件可以从 [这里](https://download.openmmlab.com/mmselfsup/tutorial/epoch_1.pth) 下载，存放在文件夹 `mmselfsup/work_dirs/selfsup/simclr/simclr_resnet50_1xb32-coslr-1e_tinyin200` 里。

In [ ]:
%cd /content/mmselfsup
!mkdir -p work_dirs/selfsup/simclr/simclr_resnet50_1xb32-coslr-1e_tinyin200
!wget https://download.openmmlab.com/mmselfsup/tutorial/epoch_1.pth  -P  work_dirs/selfsup/simclr/simclr_resnet50_1xb32-coslr-1e_tinyin200

可以使用命令 `tools/model_converters/extract_backbone_weights.py` 来提取自监督预训练模型的 backbone 权值，代码如下：

```python
python tools/model_converters/extract_backbone_weights.py {CHECKPOINT} {MODEL_FILE}
```

参数:
- CHECKPOINT：自监督预训练过程中保存下来（名为 `epoch_*.pth`）的模型文件路径
- MODEL_FILE：输出 backbone 权重文件的保存路径。

In [ ]:
%cd /content/mmselfsup

In [ ]:
!python ./tools/model_converters/extract_backbone_weights.py \
work_dirs/selfsup/simclr/simclr_resnet50_1xb32-coslr-1e_tinyin200/epoch_1.pth \
work_dirs/selfsup/simclr/simclr_resnet50_1xb32-coslr-1e_tinyin200/backbone.pth

### 3.2 针对 MMSelfSup 模型库里的模型文件

**注意：MMSelfSup 的 [模型库](https://github.com/open-mmlab/mmselfsup/blob/master/docs/en/model_zoo.md) 中的模型文件都已经提取过 backbone 权值，不需要再次提取！**我们直接使用即可。

在模型库中找到在 SimCLR 的预训练模型文件 `simclr_resnet50_8xb32-coslr-200e_in1k`，下载放在 `checkpoints` 文件夹里 

In [ ]:
%cd /content/mmselfsup
!mkdir checkpoints
!wget https://download.openmmlab.com/mmselfsup/simclr/simclr_resnet50_8xb32-coslr-200e_in1k_20220428-46ef6bb9.pth -P checkpoints

## 4. 自监督预训练模型的 “检测” 下游任务评估

### 4.1 写自监督预训练模型的 “检测” 评估配置文件

1. 新建一个名为 `faster_rcnn_r50_c4_mstrain_3k_voc0712.py` 的配置文件。（配置文件命名要求 & 含义可参考[这里](https://mmsegmentation.readthedocs.io/zh_CN/latest/tutorials/config.html#id3))



2. `faster_rcnn_r50_c4_mstrain_3k_voc0712.py` 训练配置文件的内容：
    1. 继承 [faster_rcnn_r50_c4_mstrain_24k_voc0712.py](https://github.com/open-mmlab/mmselfsup/blob/master/configs/benchmarks/mmdetection/voc0712/faster_rcnn_r50_c4_mstrain_24k_voc0712.py) 配置文件
    2. 根据 batch size 调整学习率（调整原则请参考：[这里](https://mmselfsup.readthedocs.io/zh_CN/latest/get_started.html#id2)）
    3. 根据需求修改参数：模型训练多少个 iteration 评估一次、模型训练多少个 iteration 保存一次 checkpoint 文件 以及 总共训练多少个 iteration

In [ ]:
%%writefile /content/mmselfsup/configs/benchmarks/mmdetection/voc0712/faster_rcnn_r50_c4_mstrain_3k_voc0712.py
_base_ = 'faster_rcnn_r50_c4_mstrain_24k_voc0712.py'

optimizer = dict(
    lr=0.02 * (1 / 8)
)

evaluation = dict(interval=1000, metric='mAP')

checkpoint_config = dict(by_epoch=False, interval=1000)

runner = dict(type='IterBasedRunner', max_iters=3000)

### 4.2 开始 “检测” 下游任务的评估

在运行 “检测” 下游任务评估时，我们使用 [mim_dist_train_c4.sh](https://github.com/open-mmlab/mmselfsup/blob/master/tools/benchmarks/mmdetection/mim_dist_train_c4.sh) 脚本来启动训练。

```shell
bash tools/benchmarks/mmdetection/mim_dist_train_c4.sh ${CONFIG} ${PRETRAIN} ${GPUS}
```

参数：
- CONFIG：“检测” 评估所使用的配置文件，位于 configs/benchmarks/mmdetection/ 里对应的数据集目录下
- PRETRAIN: 自监督预训练模型的 backbone 权重文件所在的路径。

In [ ]:
%cd /content/mmselfsup

In [ ]:
!bash tools/benchmarks/mmdetection/mim_dist_train_c4.sh \
configs/benchmarks/mmdetection/voc0712/faster_rcnn_r50_c4_mstrain_3k_voc0712.py \
checkpoints/simclr_resnet50_8xb32-coslr-200e_in1k_20220428-46ef6bb9.pth \
1